In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
mnist=input_data.read_data_sets("MNIST_data",one_hot=True)

#每个批次的大小
batch_size=50
#计算一共有多少个批次
a_batch=mnist.train.num_examples//batch_size

#初始化权重
def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

#初始化偏置
def bias_variable(shape):
    initial=tf.constant(0,1,shape=shape)
    return tf.Variable(initial)

#卷积层
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],
                          strides=[1,2,2,1],padding="SAME")

#定义两个placeholder
x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])

x_image=tf.reshape(x,[-1,28,28,1])

#初始化第一个卷积层的权重和偏置
W_conv1=weight_variable([5,5,1,6])
#5*5的采样窗口，32个卷积核从一个平面抽取特征
b_conv1=bias_variable([6])
#池化
h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1=max_pool_2x2(h_conv1)

W_conv2=weight_variable([5,5,6,6])
b_conv2=bias_variable([6])
h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2=max_pool_2x2(h_conv2)
#28*28的图片第一次卷积后是28*28，第一次池化后是14*14
#第二次卷积后是14*14，第二次池化后是7*7

#定义第一个全连接层的权重
W_fe1=weight_variable([7*7*6,50])
b_fe1=bias_variable([50])

#把池化层2的输出扁平化
h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*6])
h_fel=tf.nn.relu(tf.matmul(h_pool2_flat,W_fe1)+b_fe1)

keep_prob=tf.placeholder(tf.float32)
h_fe1_drop=tf.nn.dropout(h_fel,keep_prob)

#定义第二个全连接层
W_fe2=weight_variable([50,10])
b_fe2=bias_variable([10])

#计算输出
prediction=tf.nn.softmax(tf.matmul(h_fe1_drop,W_fe2)+b_fe2)

#交叉熵代价函数
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits
                             (labels=y,logits=prediction))
#优化
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在一个布尔型列表中
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
#求准确率
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer=tf.summary.FileWriter("logs/",sess.graph)
    for epoch in range(11):
        for batch in range(a_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,
                                           keep_prob:0.7})
     
        acc=sess.run(accuracy,feed_dict={x:mnist.test.images,
                                         y:mnist.test.labels
                                         ,keep_prob:1.0})
        print('iter',str(epoch),'testing Accuracy',str(acc))
          

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
iter 0 testing Accuracy 0.8567
iter 1 testing Accuracy 0.9003
iter 2 testing Accuracy 0.9143
iter 3 testing Accuracy 0.9243
iter 4 testing Accuracy 0.9294
iter 5 testing Accuracy 0.9364
iter 6 testing Accuracy 0.9395
iter 7 testing Accuracy 0.9445
iter 8 testing Accuracy 0.9472
iter 9 testing Accuracy 0.9473
iter 10 testing Accuracy 0.9517
